# Accurate LPI Radar Waveform Recognition With CWD-TFA for Deep Convolutional Network


### 1 Setup

#### 1.1 Imports

In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import h5py
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
)
import cv2
import seaborn as sns
import os
import numpy as np
from scipy.fft import fft, ifft, fftshift

#### 1.2 Device Selection

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

#### 1.3 Data Input

In [ ]:
def load_spectrogram_h5s(root_folder, mod_types):
    """
    Loads all .h5 spectrogram files from the specified modulation types
    and returns a dict structured as:
      {
          "lfm_up": {
              0: <np.ndarray>,
              1: <np.ndarray>,
              ...
          },
          ...
      }

    Parameters:
    - root_folder (str): Path to the top-level "spectrograms" directory.
    - mod_types (list): List of modulation families to include, e.g. ['FM', 'PM']

    Returns:
    - Dictionary containing all spectrograms, indexed by modulation name and integer index.
    """
    spectrogram_dict = {}

    for mod_type in mod_types:
        mod_path = os.path.join(root_folder, mod_type)
        if not os.path.exists(mod_path):
            print(f"⚠️ Warning: {mod_path} does not exist. Skipping.")
            continue

        print(f"📂 Loading from {mod_type}...")
        files = [f for f in os.listdir(mod_path) if f.endswith(".h5")]

        for file in tqdm(files, desc=f"   {mod_type}", unit="file"):
            mod_name = file[:-3]  # Strip '.h5'
            file_path = os.path.join(mod_path, file)
            spectrogram_dict[mod_name] = {}

            try:
                with h5py.File(file_path, "r") as h5f:
                    for key in h5f.keys():
                        idx = int(key)  # Convert string index to int
                        spectrogram_dict[mod_name][idx] = np.array(h5f[key])
            except Exception as e:
                print(f"❌ Failed to load {file_path}: {e}")

    return spectrogram_dict


In [ ]:
# Ensure the main directory exists
# Add in path to where the spectrograms are stored
data_path = "C:/Users/scocks/Documents/hehehehehehhe/images/"
os.makedirs(data_path, exist_ok=True)

img_res = 224
img_count = 1000
snr = 0


folder_name = f"General_Images_res_{img_res}_sz_{img_count}_SNR_{snr}"
folder_path = data_path+folder_name

modulation_types = [
    "FM",
    "PM",
    "HYBRID",
]

data = load_spectrogram_h5s(folder_path, modulation_types)

In [ ]:
def convert_spectrogram_dict_to_xy(data_dict):
    """
    Converts a dictionary of spectrograms into (X, y) format for ML.

    Parameters:
    - data_dict: Output from load_spectrogram_h5s(), e.g.
        {
            "lfm_up": {0: np.array, 1: np.array, ...},
            "bpsk":   {0: np.array, ...},
            ...
        }

    Returns:
    - X: np.ndarray of shape (N, H, W, C)
    - y: np.ndarray of shape (N,) with string labels like 'lfm_up'
    """
    X_list = []
    y_list = []

    for label, spectros in data_dict.items():
        for idx in sorted(spectros.keys()):
            X_list.append(spectros[idx])
            y_list.append(label)

    X = np.array(X_list)
    y = np.array(y_list)

    return X, y


In [ ]:
X, y = convert_spectrogram_dict_to_xy(data)

In [ ]:
print(X.shape)
print(y.shape)

#### 1.4 Label Encoder

In [ ]:
label_encoder = LabelEncoder()

#### 1.5 Data Loader

In [ ]:
def prepare_dataloader(X, y, batch_size=32, shuffle=False, num_workers=2, device="cpu"):
    # Convert NumPy arrays to PyTorch tensors
    if isinstance(X, np.ndarray):
        X = torch.tensor(X, dtype=torch.float32)
    elif not isinstance(X, torch.Tensor):
        raise TypeError("Input X must be a NumPy array or PyTorch tensor")

    if isinstance(y, np.ndarray):
        y = torch.tensor(y, dtype=torch.long)
    elif not isinstance(y, torch.Tensor):
        raise TypeError("Labels y must be a NumPy array or PyTorch tensor")

    # Ensure X has four dimensions (N, C, H, W)
    if X.ndim == 3:  # If (N, H, W), add a channel dimension
        X = X.unsqueeze(1)  # (N, 1, H, W)
    elif X.ndim == 4 and X.shape[-1] in [1, 3]:  # (N, H, W, C) case
        X = X.permute(0, 3, 1, 2)  # Convert to (N, C, H, W)

    # Move data to the correct device
    X, y = X.to(device), y.to(device)

    # Create dataset and dataloader
    dataset = TensorDataset(X, y)
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=(device == "cuda"),
    )

    return loader

### 2 Pre-processing

In [ ]:
def pre_processing(images):
    # Resize images to 128x128 using bicubic interpolation
    resized_images = np.array(
        [cv2.resize(img, (64, 64), interpolation=cv2.INTER_CUBIC) for img in images]
    )

    # Convert to grayscale using luminance formula
    # Assumes images are in RGB format
    grayscale_images = np.dot(resized_images[...,:3], [0.2989, 0.5870, 0.1140])

    return grayscale_images


In [ ]:
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_pre_processed = pre_processing(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_pre_processed, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

In [ ]:
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

print("Class Distribution in Training Set:")
for label, count in zip(unique_train, counts_train):
    print(f"Class {label}: {count} samples")

print("\nClass Distribution in Test Set:")
for label, count in zip(unique_test, counts_test):
    print(f"Class {label}: {count} samples")

#### 2.1 Pre-processing Display

In [ ]:
# selected_display = np.random.randint(len(X_train))
selected_display = 0

In [ ]:
print("Train Images shape:", X_pre_processed.shape)
print("Train Metadata shape:", y_encoded.shape)
print(f"{y[selected_display]} = {y_encoded[selected_display]}")

In [ ]:
plt.imshow(X[selected_display])
plt.show()

In [ ]:
plt.imshow(X_pre_processed[selected_display], cmap="grey")
plt.show()
print(X_pre_processed[selected_display].shape)


### 3 Training

#### 3.1 Training Setup

In [ ]:
def train_model(
    model,
    train_loader,
    device,
    criterion,
    optimizer,
    scheduler=None,  # Added scheduler as an optional argument
    epochs=10,
    patience=3,
    min_delta=0.0,
):
    model.to(device)
    model.train()

    loss_history = []
    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(epochs):
        total_loss = 0.0

        # Progress bar for each epoch
        progress_bar = tqdm(
            train_loader,
            desc=f"Epoch {epoch+1}/{epochs}",
            leave=True,
            dynamic_ncols=True,
        )

        for inputs, labels in progress_bar:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output_class = model(inputs)  # Forward pass

            loss = criterion(output_class, labels)  # Compute loss

            loss.backward()  # Backpropagation

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # 🔥 Prevent exploding gradients

            optimizer.step()  # Apply gradients

            total_loss += loss.item()

            # Live loss display
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        # Average loss for the epoch
        avg_loss = total_loss / len(train_loader)
        loss_history.append(avg_loss)
        print(f"Epoch {epoch+1} average loss: {avg_loss:.4f}")

        # Step the scheduler if provided
        if scheduler:
            scheduler.step()

        # Early stopping
        if avg_loss < best_loss - min_delta:
            best_loss = avg_loss
            patience_counter = 0  # Reset counter if improvement is seen
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break  # Stop training early

    return loss_history

##### Loss Curve

In [ ]:
def plot_loss_curve(loss_history, title="Training Loss Over Epochs"):
    epochs = len(loss_history)

    plt.figure(figsize=(8, 5))
    plt.plot(range(1, epochs + 1), loss_history, marker="o", label="Training Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()

##### Conf Matirx

In [ ]:
def display_confusion_matrix(model, dataloader, device, class_names):
    """
    Generate and display a confusion matrix for a trained model.
    
    Parameters:
        model (torch.nn.Module): Trained PyTorch model.
        dataloader (torch.utils.data.DataLoader): DataLoader for evaluation dataset.
        device (torch.device): Device to run evaluation on (CPU/GPU).
        class_names (list): List of class names corresponding to the output classes.
    """
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)  # Get class predictions

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    cm_normalized = cm.astype(np.float32) / cm.sum(axis=1, keepdims=True)  # Normalize per row

    # Plot confusion matrix
    num_classes = len(class_names)
    plt.figure(figsize=(max(12, num_classes * 0.8), max(10, num_classes * 0.6)))  # Dynamic size based on num classes
    sns.heatmap(cm_normalized, annot=True, fmt=".1f", cmap="Blues",  # Use 1 decimal place
                xticklabels=class_names, yticklabels=class_names,
                annot_kws={"size": max(8, 12 - num_classes // 5)},  # Adjust font size dynamically
                cbar_kws={'label': 'Normalized Score'})
    plt.xlabel("Predicted Labels", fontsize=12)
    plt.ylabel("True Labels", fontsize=12)
    plt.title("Confusion Matrix", fontsize=14)
    plt.xticks(rotation=45, ha='right')  # Rotate x labels for better readability
    plt.yticks(rotation=0)
    plt.tight_layout()  # Adjust layout to prevent clipping
    plt.show()

#### 3.2 Model

In [ ]:
class module_thing(nn.Module):
    def __init__(self):
        super(module_thing, self).__init__()

        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1,3), stride=1,padding=(0,1))
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), stride=1,padding=(1,0))

        self.conv3 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1,3), stride=1,padding=(0,1))
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), stride=1,padding=(1,0))

        self.conv6 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1, stride=1)

    def forward(self, x):

        x = self.pool1(x)
        x0 = x

        x1 = self.conv1(x)
        x2 = self.conv2(x)

        x_cat1 = torch.cat((x1,x2), dim=1)
        x  = self.conv3(x_cat1)

        x1 = self.conv4(x)
        x2 = self.conv5(x)

        x = torch.cat((x1,x2,x_cat1), dim=1)

        x = self.conv6(x)

        x+=x0

        return x

##### Main Model

In [ ]:
class MainModel(nn.Module):
    def __init__(self, num_classes):
        super(MainModel, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=64, kernel_size=7, stride=1, padding=0
        )

        self.m1 = module_thing()
        self.m2 = module_thing()
        self.m3 = module_thing()

        self.gap = nn.AdaptiveAvgPool2d(output_size=(1,1))

        self.fc1= nn.Linear(in_features=64, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=num_classes)

        self.dropout = nn.Dropout(p=0.5)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        x = self.conv1(x)

        x = self.m1(x)
        x = self.m2(x)
        x = self.m3(x)

        x = self.gap(x)
        x = torch.flatten(x,1)

        x = self.fc1(x)
        x = self.dropout(x)
        output_class = self.fc2(x)

        # output_class = self.softmax(output_class)

        return output_class

##### Model Summary

In [ ]:
model = MainModel(6)
summary(model , input_size=(1, 1, 64, 64))

#### 3.3 Actual Training

In [ ]:
# Prepare DataLoaders
train_loader = prepare_dataloader(
    X_train,
    y_train,
    batch_size=32,
    shuffle=True,
)

In [ ]:
num_classes = len(np.unique(y_encoded))

model = MainModel(num_classes=num_classes).to(device)


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
learning_rate = 1e-2
optimizer = optim.SGD(
    model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001
)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=45, gamma=0.1)


In [ ]:
# Train the model
epoch_count = 1
loss_history = train_model(
    model=model,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    epochs=epoch_count,
    patience=50,
)

In [ ]:
plot_loss_curve(loss_history)

In [ ]:
display_confusion_matrix(model, train_loader, device,np.unique(y_train))

#### 3.4 Save Model to File

In [ ]:
mds = modulation_types[0]
if len(modulation_types) == 3:
    mds = "ALL"

model_file_name = f"LPI_Net_model_e{epoch_count}_lr{learning_rate}_snr_{snr}_mds_{mds}"

In [ ]:
torch.save(model, model_file_name + ".pth")

### 4 Testing

In [ ]:
def evaluate_model(model, test_loader, label_encoder, device):
    """
    Evaluates the trained model and displays accuracy, normalized confusion matrix, 
    F1-score, and one output image per class.

    Args:
        model: Trained PyTorch model.
        test_loader: DataLoader for test set.
        label_encoder: Label encoder to decode class names.
        device: 'cuda' or 'cpu' where evaluation happens.
    """
    import torch
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import confusion_matrix, classification_report

    model.to(device)
    model.eval()  # Set model to evaluation mode

    y_true = []
    y_pred = []
    correct = 0
    total = 0

    class_images = {}  # Store one output image per class

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Get model predictions (ONLY output_class)
            output_class = model(inputs)  

            # Get predicted class (argmax over logits)
            preds = torch.argmax(output_class, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            y_true.extend(labels.cpu().tolist())
            y_pred.extend(preds.cpu().tolist())

            # Store one output image per class
            for img, pred_class in zip(inputs, preds):
                pred_class = pred_class.item()
                if pred_class not in class_images:
                    class_images[pred_class] = img.detach().cpu()

    # Compute Accuracy
    accuracy = 100 * correct / total
    print(f"✅ Accuracy: {accuracy:.2f}%")

    # Decode class names
    class_names = label_encoder.inverse_transform(range(len(set(y_true))))

    # Compute & Normalize Confusion Matrix (%)
    cm = confusion_matrix(y_true, y_pred)
    cm_percentage = cm.astype(float) / cm.sum(axis=1, keepdims=True) * 100  # Convert to percentages

    # Plot Confusion Matrix (Percentage)
    num_classes = len(class_names)
    plt.figure(figsize=(max(12, num_classes * 0.8), max(10, num_classes * 0.6)))  # Dynamic size based on num classes
    sns.heatmap(cm_percentage, annot=True, fmt=".1f", cmap="Blues",  # Use 1 decimal place
                xticklabels=class_names, yticklabels=class_names,
                annot_kws={"size": max(8, 12 - num_classes // 5)},  # Adjust font size dynamically
                cbar_kws={'label': 'Percentage'})
    plt.xlabel("Predicted Labels", fontsize=12)
    plt.ylabel("True Labels", fontsize=12)
    plt.title("Confusion Matrix (Percentage)", fontsize=14)
    plt.xticks(rotation=45, ha='right')  # Rotate x labels for better readability
    plt.yticks(rotation=0)
    plt.tight_layout()  # Adjust layout to prevent clipping
    plt.show()

    # Print Classification Report (Precision, Recall, F1-score)
    print("\n✅ Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Prepare DataLoaders
test_loader = prepare_dataloader(
    X_test,
    y_test,
    batch_size=32,
)

In [ ]:
# Evaluate the model
evaluate_model(model, test_loader, label_encoder, device)